# RiskEngine End-to-End (30–180d history → 90d risk)
This notebook builds a risk prediction pipeline over your four CSVs.

**Inputs expected:**
- `Patient_info.csv`
- `Biochemical_parameters.csv`
- `Diagnostics.csv`
- `Glucose_measurements.csv` (large; we'll aggregate to daily metrics)

**Outputs:**
- `outputs/feature_dataset.parquet` – modeling dataset with rolling-window features
- `outputs/model_calibrated.joblib` – trained + calibrated model
- `outputs/metrics.json` – AUROC, AUPRC, Brier, confusion matrix
- `outputs/plots/` – ROC, PR, Calibration plots


In [2]:
# --- Setup ---
import os, json, math, gc, sys, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

# Prefer xgboost; fallback to HistGradientBoosting if not available
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    from sklearn.ensemble import HistGradientBoostingClassifier
    HAS_XGB = False

try:
    import shap
    HAS_SHAP = True
except Exception:
    HAS_SHAP = False

BASE = Path(".").resolve()
OUT = BASE / "outputs"
PLOTS = OUT / "plots"
for p in [OUT, PLOTS]:
    p.mkdir(parents=True, exist_ok=True)

print("Using XGBoost:", HAS_XGB, "| SHAP available:", HAS_SHAP)


Using XGBoost: True | SHAP available: True


In [3]:
# --- Config: set your file paths here ---
PATH_PATIENT_INFO = "Patient_info.csv"
PATH_LABS = "Biochemical_parameters.csv"
PATH_DIAG = "Diagnostics.csv"
PATH_CGM = "Glucose_measurements.csv"   # large file

# CGM cadence assumption (15-min ~ 96/day)
EXPECTED_READS_PER_DAY = 96

# Rolling windows (in days)
WINDOWS = [30, 90, 180]

# Label thresholds
FUTURE_90D_HYPER_MEAN_THRESHOLD = 0.35   # avg future 90d fraction > 180 mg/dL
A1C_RISE_THRESHOLD = 0.5                 # A1c increase >= 0.5 absolute within 90d


In [4]:
# --- Load the three small tables ---
pi = pd.read_csv(PATH_PATIENT_INFO)
bp = pd.read_csv(PATH_LABS)
dg = pd.read_csv(PATH_DIAG)

# Parse dates in patient_info
for c in ["Initial_measurement_date","Final_measurement_date",
          "Initial_biochemical_parameters_date","Final_biochemical_parameters_date"]:
    if c in pi.columns:
        pi[c] = pd.to_datetime(pi[c], dayfirst=True, errors="coerce")

# Parse labs
if "Reception_date" in bp.columns:
    bp["Reception_date"] = pd.to_datetime(bp["Reception_date"], dayfirst=True, errors="coerce")

print("Patients:", pi['Patient_ID'].nunique(), 
      "| Labs coverage:", bp['Patient_ID'].nunique(), 
      "| Diagnostics coverage:", dg['Patient_ID'].nunique())
bp.head(3), dg.head(3), pi.head(3)


Patients: 736 | Labs coverage: 723 | Diagnostics coverage: 511


(  Patient_ID Reception_date                              Name  Value
 0  LIB193265     2018-05-09                         Potassium    4.5
 1  LIB193265     2018-05-09                   HDL cholesterol   64.0
 2  LIB193265     2018-05-09  Gamma-glutamyl Transferase (GGT)   11.0,
   Patient_ID    Code                                        Description
 0  LIB193263   272.4               Other and unspecified hyperlipidemia
 1  LIB193264   354.0                             Carpal tunnel syndrome
 2  LIB193264  574.00  Calculus of gallbladder with acute cholecystit...,
   Patient_ID Sex  Birth_year Initial_measurement_date Final_measurement_date  \
 0  LIB193263   M        1965               2020-09-06             2022-03-19   
 1  LIB193264   F        1975               2020-10-06             2022-03-19   
 2  LIB193265   F        1980                      NaT             2022-03-19   
 
    Number_of_days_with_measures  Number_of_measurements  \
 0                           648        

In [5]:
def aggregate_cgm_daily(path_cgm, expected_reads_per_day=96, chunksize=2_000_000):
    import pandas as pd, numpy as np, math, gc

    required_cols = ["Patient_ID","Measurement_date","Measurement_time","Measurement"]
    usecols = required_cols  # restrict to just the needed columns
    dtypes  = {"Patient_ID":"string", "Measurement":"float32"}

    agg = {}

    for chunk in pd.read_csv(
        path_cgm,
        usecols=usecols,
        dtype=dtypes,
        chunksize=chunksize,
        low_memory=True,
        on_bad_lines="skip",           # skip malformed lines
        encoding_errors="replace",     # be forgiving on encoding
        na_values=["", "NA", "NaN", "null", "None"]
    ):
        # normalize column names (in case of stray whitespace)
        chunk.columns = [c.strip() for c in chunk.columns]

        # coerce date; keep only valid rows with a numeric measurement and patient id
        chunk["Measurement_date"] = pd.to_datetime(
            chunk["Measurement_date"], dayfirst=True, errors="coerce"
        ).dt.date

        chunk = chunk.dropna(subset=["Patient_ID","Measurement_date","Measurement"])
        if chunk.empty:
            continue

        # flags
        chunk["gt_180"] = (chunk["Measurement"] > 180).astype("int16")
        chunk["lt_70"]  = (chunk["Measurement"] < 70).astype("int16")

        # group by patient/day
        grp = (chunk
               .groupby(["Patient_ID","Measurement_date"], as_index=False)
               .agg(n_reads=("Measurement","size"),
                    mean_glu=("Measurement","mean"),
                    std_glu=("Measurement","std"),
                    max_glu=("Measurement","max"),
                    min_glu=("Measurement","min"),
                    gt_180_sum=("gt_180","sum"),
                    lt_70_sum=("lt_70","sum"))
              )

        # ignore any accidental zero-read groups (shouldn't exist, but safe)
        grp = grp[grp["n_reads"] > 0]
        if grp.empty:
            continue

        # accumulate
        for _, r in grp.iterrows():
            key = (r["Patient_ID"], r["Measurement_date"])
            n   = int(r["n_reads"])
            mu  = float(r["mean_glu"])
            sd  = float(0.0 if pd.isna(r["std_glu"]) else r["std_glu"])

            if key not in agg:
                agg[key] = {
                    "Patient_ID": r["Patient_ID"],
                    "date": pd.to_datetime(str(r["Measurement_date"])),
                    "n_reads": n,
                    "sum_glu": mu * n,
                    "sum_sq_glu": (sd*sd + mu*mu) * n,
                    "max_glu": float(r["max_glu"]),
                    "min_glu": float(r["min_glu"]),
                    "gt_180": int(r["gt_180_sum"]),
                    "lt_70": int(r["lt_70_sum"]),
                }
            else:
                a = agg[key]
                a["n_reads"] += n
                a["sum_glu"] += mu * n
                a["sum_sq_glu"] += (sd*sd + mu*mu) * n
                a["max_glu"] = max(a["max_glu"], float(r["max_glu"]))
                a["min_glu"] = min(a["min_glu"], float(r["min_glu"]))
                a["gt_180"]  += int(r["gt_180_sum"])
                a["lt_70"]   += int(r["lt_70_sum"])

        del chunk, grp
        gc.collect()

    # materialize
    rows = []
    for (pid, day), a in agg.items():
        n = max(1, a["n_reads"])
        mean = a["sum_glu"] / n
        var  = max(0.0, a["sum_sq_glu"]/n - mean*mean)
        std  = math.sqrt(var)
        rows.append({
            "Patient_ID": pid,
            "date": a["date"],
            "n_reads": a["n_reads"],
            "coverage": min(1.0, a["n_reads"] / expected_reads_per_day),
            "mean_glu": mean,
            "std_glu": std,
            "max_glu": a["max_glu"],
            "min_glu": a["min_glu"],
            "frac_gt_180": a["gt_180"] / n,
            "frac_lt_70": a["lt_70"] / n,
            "tir_70_180": max(0.0, 1.0 - (a["gt_180"] + a["lt_70"]) / n),
        })

    daily = pd.DataFrame(rows).sort_values(["Patient_ID","date"]).reset_index(drop=True)
    # enforce basic quality filters here so no 0-read rows survive
    daily = daily[(daily["n_reads"] > 0)]
    return daily


In [6]:
daily_cgm = aggregate_cgm_daily(PATH_CGM, expected_reads_per_day=EXPECTED_READS_PER_DAY)
daily_cgm.to_parquet(OUT / "daily_cgm.parquet", index=False)
print(daily_cgm.shape)
daily_cgm.head(10)


(184269, 11)


,Patient_ID,date,n_reads,coverage,mean_glu,std_glu,max_glu,min_glu,frac_gt_180,frac_lt_70,tir_70_180
0,LIB193263,2020-01-07,90,0.937500,135.677780,41.512100,244.0,77.0,0.133333,0.000000,0.866667
1,LIB193263,2020-01-08,96,1.000000,173.541672,61.597149,356.0,64.0,0.322917,0.020833,0.656250
2,LIB193263,2020-01-09,96,1.000000,214.708328,63.121422,331.0,98.0,0.666667,0.000000,0.333333
3,LIB193263,2020-01-10,92,0.958333,149.858688,35.584778,242.0,80.0,0.173913,0.000000,0.826087
4,LIB193263,2020-01-11,92,0.958333,130.076080,38.457409,211.0,82.0,0.130435,0.000000,0.869565
5,LIB193263,2020-01-12,94,0.979167,172.765961,111.982666,382.0,43.0,0.436170,0.223404,0.340426
6,LIB193263,2020-02-07,96,1.000000,168.666672,48.101574,262.0,55.0,0.416667,0.052083,0.531250
7,LIB193263,2020-02-08,81,0.843750,157.629623,35.000160,248.0,99.0,0.209877,0.000000,0.790123
8,LIB193263,2020-02-09,94,0.979167,212.308517,24.608778,257.0,164.0,0.829787,0.000000,0.170213
9,LIB193263,2020-02-10,91,0.947917,169.208786,53.145191,289.0,57.0,0.274725,0.054945,0.670330


In [7]:
# --- Quick load of pre-aggregated daily CGM if you've already generated it ---
daily_path = OUT / "daily_cgm.parquet"
if daily_path.exists():
    daily_cgm = pd.read_parquet(daily_path)
    print("Loaded daily CGM:", daily_cgm.shape)
    display(daily_cgm.head())
else:
    print("Daily CGM parquet not found. Generate it in the previous cell when you have the big file locally.")


Loaded daily CGM: (184269, 11)


,Patient_ID,date,n_reads,coverage,mean_glu,std_glu,max_glu,min_glu,frac_gt_180,frac_lt_70,tir_70_180
0,LIB193263,2020-01-07,90,0.937500,135.677780,41.512100,244.0,77.0,0.133333,0.000000,0.866667
1,LIB193263,2020-01-08,96,1.000000,173.541672,61.597149,356.0,64.0,0.322917,0.020833,0.656250
2,LIB193263,2020-01-09,96,1.000000,214.708328,63.121422,331.0,98.0,0.666667,0.000000,0.333333
3,LIB193263,2020-01-10,92,0.958333,149.858688,35.584778,242.0,80.0,0.173913,0.000000,0.826087
4,LIB193263,2020-01-11,92,0.958333,130.076080,38.457409,211.0,82.0,0.130435,0.000000,0.869565


In [8]:
# --- Prepare A1c time series per patient (for features + label proxy) ---
# We filter labs to A1c and keep (Patient_ID, date, value)
a1c = bp[bp["Name"].str.contains("Glycated hemoglobin", case=False, na=False)].copy()
a1c = a1c.rename(columns={"Reception_date":"date","Value":"a1c"})
a1c = a1c[["Patient_ID","date","a1c"]].dropna()
a1c = a1c.sort_values(["Patient_ID","date"])
print("A1c rows:", a1c.shape)
display(a1c.head())


A1c rows: (1963, 3)


,Patient_ID,date,a1c
11,LIB193265,2018-05-09,8.6
33,LIB193265,2019-12-03,7.5
60,LIB193265,2020-02-06,7.3
165,LIB193266,2021-05-04,7.5
169,LIB193266,2021-05-11,7.5


In [9]:
# dg is your Diagnostics.csv as a DataFrame
# expect columns like: Patient_ID, Code (ICD), maybe others
top_codes = dg["Code"].value_counts().head(15).index.tolist()

diag_flags = (
    dg.loc[dg["Code"].isin(top_codes), ["Patient_ID","Code"]]
      .assign(val=1)
      .pivot_table(index="Patient_ID", columns="Code", values="val", aggfunc="max", fill_value=0)
)

# prefix and tidy
diag_flags.columns = [f"diag_{c}" for c in diag_flags.columns]
diag_flags = diag_flags.reset_index()

print("diag_flags shape:", diag_flags.shape)
display(diag_flags.head())


diag_flags shape: (297, 16)


,Patient_ID,diag_244.9,diag_250.4,diag_250.5,diag_250.51,diag_250.6,diag_272.0,diag_272.4,diag_278.00,diag_354.0,diag_362.01,diag_401.9,diag_477.0,diag_477.9,diag_493.9,diag_733.00
0,LIB193263,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,LIB193264,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,LIB193272,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,LIB193273,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,LIB193274,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0


In [10]:
# Make sure every Patient_ID in daily_cgm appears in diag_flags with 0s
all_pids = pd.DataFrame({"Patient_ID": daily_cgm["Patient_ID"].unique()})
diag_cols = [c for c in diag_flags.columns if c.startswith("diag_")]

# left-join and fill missing diag_* with 0
diag_flags = all_pids.merge(diag_flags, on="Patient_ID", how="left")
for c in diag_cols:
    if c in diag_flags.columns:
        diag_flags[c] = diag_flags[c].fillna(0).astype(int)
    else:
        # if a top code column is completely missing, create it
        diag_flags[c] = 0

# keep only Patient_ID + diag_* columns
keep_cols = ["Patient_ID"] + diag_cols
diag_flags = diag_flags[keep_cols].copy()

print("diag_flags (padded) shape:", diag_flags.shape)
diag_flags.head()


diag_flags (padded) shape: (734, 16)


,Patient_ID,diag_244.9,diag_250.4,diag_250.5,diag_250.51,diag_250.6,diag_272.0,diag_272.4,diag_278.00,diag_354.0,diag_362.01,diag_401.9,diag_477.0,diag_477.9,diag_493.9,diag_733.00
0,LIB193263,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,LIB193264,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,LIB193265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,LIB193266,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LIB193267,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
print("LEFT daily_feat (for asof):")
print(daily_feat[["Patient_ID","index_date"]].head(15))
print(daily_feat[["Patient_ID","index_date"]].tail(15))
print("dtype:", daily_feat["index_date"].dtype)

print("\nRIGHT a1c_prior_tbl:")
print(a1c_prior_tbl[["Patient_ID","a1c_date"]].head(15))
print(a1c_prior_tbl[["Patient_ID","a1c_date"]].tail(15))
print("dtype:", a1c_prior_tbl["a1c_date"].dtype)

# extra check: monotonicity within each Patient_ID
print("\nNon-monotonic counts LEFT:",
      (daily_feat.groupby("Patient_ID")["index_date"].diff() < pd.Timedelta(0)).sum())
print("Non-monotonic counts RIGHT:",
      (a1c_prior_tbl.groupby("Patient_ID")["a1c_date"].diff() < pd.Timedelta(0)).sum())


LEFT daily_feat (for asof):


NameError: name 'daily_feat' is not defined

In [ ]:
# ================================
# Build rolling-window features + labels (no-merge_asof version)
# ================================
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# ---------- Guards ----------
if "daily_cgm" not in globals():
    raise RuntimeError("daily_cgm not loaded. Create or load outputs/daily_cgm.parquet first.")
if "pi" not in globals():
    raise RuntimeError("pi (Patient_info) not loaded.")
if "a1c" not in globals():
    a1c = pd.DataFrame(columns=["Patient_ID","date","a1c"])
if "diag_flags" not in globals():
    diag_flags = pd.DataFrame(columns=["Patient_ID"])

# ---------- Utils ----------
def _ensure_dt_naive(dt_series: pd.Series) -> pd.Series:
    s = pd.to_datetime(dt_series, errors="coerce")
    if getattr(s.dtype, "tz", None) is not None:
        s = s.dt.tz_convert(None)
    return s

# ---------- Prep inputs ----------
daily_cgm = daily_cgm.copy()
daily_cgm["Patient_ID"] = daily_cgm["Patient_ID"].astype("string")
daily_cgm["date"]       = _ensure_dt_naive(daily_cgm["date"])

a1c = a1c.copy()
if len(a1c):
    a1c["Patient_ID"] = a1c["Patient_ID"].astype("string")
    a1c["date"]       = _ensure_dt_naive(a1c["date"])
    a1c = a1c.dropna(subset=["Patient_ID","date","a1c"]).sort_values(["Patient_ID","date"]).reset_index(drop=True)

# ---------- Rolling features ----------
def add_rolling_features(df, cols, windows):
    df = df.sort_values(["Patient_ID","date"]).copy()
    g = df.groupby("Patient_ID", sort=False)
    for w in windows:
        for c in cols:
            df[f"{c}_mean_{w}"] = g[c].transform(lambda s: s.rolling(window=w, min_periods=7).mean())
            df[f"{c}_std_{w}"]  = g[c].transform(lambda s: s.rolling(window=w, min_periods=7).std())
        def slope_rolling(s):
            out = np.full(len(s), np.nan, dtype="float64")
            for i in range(len(s)):
                start = max(0, i - w + 1)
                window = s.iloc[start:i+1].dropna()
                if len(window) >= 7:
                    t = np.arange(len(window)).reshape(-1,1)
                    lr = LinearRegression().fit(t, window.values)
                    out[i] = float(lr.coef_[0])
            return pd.Series(out, index=s.index)
        df[f"mean_glu_slope_{w}"] = g["mean_glu"].transform(slope_rolling)
    return df

feat_cols = ["mean_glu","std_glu","tir_70_180","frac_gt_180","frac_lt_70","coverage","n_reads"]
daily_feat = add_rolling_features(daily_cgm, feat_cols, WINDOWS)

# ---------- Index date & demographics ----------
daily_feat["index_date"] = _ensure_dt_naive(daily_feat["date"]) + pd.Timedelta(days=1)
pi_dem = pi[["Patient_ID","Sex","Birth_year"]].copy()
pi_dem["Patient_ID"] = pi_dem["Patient_ID"].astype("string")
daily_feat = daily_feat.merge(pi_dem, on="Patient_ID", how="left")

# ---------- Diagnostics ----------
diag_flags = diag_flags.copy()
diag_flags["Patient_ID"] = diag_flags["Patient_ID"].astype("string")
daily_feat = daily_feat.merge(diag_flags, on="Patient_ID", how="left")
diag_cols = [c for c in daily_feat.columns if c.startswith("diag_")]
if diag_cols:
    daily_feat[diag_cols] = daily_feat[diag_cols].fillna(0)

# ---------- A1c prior & future via searchsorted (no merge_asof) ----------
daily_feat = daily_feat.sort_values(["Patient_ID","index_date"]).reset_index(drop=True)

# Prepare outputs
daily_feat["a1c_prior"]  = np.nan
daily_feat["a1c_future"] = np.nan

if len(a1c):
    # groupwise arrays for fast lookup
    g_df  = daily_feat.groupby("Patient_ID", sort=False)
    g_a1c = a1c.groupby("Patient_ID", sort=False)

    # Iterate over patients present in daily_feat
    for pid, idx in g_df.groups.items():
        # daily index dates for this patient (sorted because daily_feat is sorted)
        idx_dates = daily_feat.loc[idx, "index_date"].to_numpy()

        # if patient has A1c records
        if pid in g_a1c.groups:
            a_idx   = g_a1c.groups[pid]
            a_dates = a1c.loc[a_idx, "date"].to_numpy()       # sorted
            a_vals  = a1c.loc[a_idx, "a1c"].to_numpy()

            # PRIOR: for each index_date, find last A1c date <= index_date
            pos = np.searchsorted(a_dates, idx_dates, side="right") - 1
            valid = pos >= 0
            prior_vals = np.full(len(idx_dates), np.nan, dtype="float64")
            prior_vals[valid] = a_vals[pos[valid]]
            daily_feat.loc[idx, "a1c_prior"] = prior_vals

            # FUTURE: next A1c within 90 days → find first A1c date >= index_date
            pos_f = np.searchsorted(a_dates, idx_dates, side="left")
            future_vals = np.full(len(idx_dates), np.nan, dtype="float64")
            in_range = (pos_f < len(a_dates))
            if in_range.any():
                # check within 90 days
                d_ok = (a_dates[pos_f[in_range]] - idx_dates[in_range]) <= np.timedelta64(90, "D")
                future_vals[in_range] = np.where(d_ok, a_vals[pos_f[in_range]], np.nan)
            daily_feat.loc[idx, "a1c_future"] = future_vals
        else:
            # no A1c for this patient → remain NaN
            pass

# ---------- Future 90d hyperglycemia burden ----------
def future_mean_next_n_days(group, col, n=90):
    s = group[col].astype(float)
    s_rev = s.iloc[::-1].rolling(window=n, min_periods=7).mean().iloc[::-1]
    return s_rev.shift(-1)

daily_feat = daily_feat.sort_values(["Patient_ID","date"]).reset_index(drop=True)
daily_feat["future90_frac_gt_180_mean"] = (
    daily_feat.groupby("Patient_ID", group_keys=False)
              .apply(lambda g: future_mean_next_n_days(g, "frac_gt_180", 90))
)

# ---------- Label ----------
daily_feat["label_90d"] = (
    (daily_feat["future90_frac_gt_180_mean"] >= FUTURE_90D_HYPER_MEAN_THRESHOLD) |
    ((daily_feat["a1c_future"] - daily_feat["a1c_prior"]) >= A1C_RISE_THRESHOLD)
).astype(int)

# ---------- Keep valid rows ----------
if "mean_glu_mean_30" not in daily_feat.columns:
    raise RuntimeError("mean_glu_mean_30 not found; rolling features may have failed.")

mask_hist   = daily_feat["mean_glu_mean_30"].notna()
mask_future = daily_feat["future90_frac_gt_180_mean"].notna()
dataset = daily_feat[mask_hist & mask_future].copy()

# ---------- Feature set ----------
feature_cols = [
    c for c in dataset.columns
    if any(tok in c for tok in ["_mean_", "_std_", "_slope_", "coverage", "n_reads"])
    and "future90" not in c
]
feature_cols += ["Birth_year"]
if "Sex" in dataset.columns:
    dataset["Sex_enc"] = dataset["Sex"].astype("category").cat.codes
    feature_cols += ["Sex_enc"]
feature_cols += [c for c in dataset.columns if c.startswith("diag_")]

keep_cols = ["Patient_ID","date","index_date","label_90d"] + feature_cols + ["a1c_prior","a1c_future"]
dataset = dataset[keep_cols].copy()
dataset = dataset.dropna(subset=feature_cols, how="all")

print("Modeling dataset shape:", dataset.shape)
display(dataset.head(3))


Modeling dataset shape: (174812, 70)


,Patient_ID,date,index_date,label_90d,n_reads,coverage,mean_glu_mean_30,mean_glu_std_30,std_glu_mean_30,std_glu_std_30,...,diag_278.00,diag_354.0,diag_362.01,diag_401.9,diag_477.0,diag_477.9,diag_493.9,diag_733.00,a1c_prior,a1c_future
6,LIB193263,2020-02-07,2020-02-08,1,96,1.000000,163.613597,28.615002,57.193871,26.466748,...,0,0,0,0,0,0,0,0,NaN,NaN
7,LIB193263,2020-02-08,2020-02-09,1,81,0.843750,162.865601,26.576687,54.419657,25.729147,...,0,0,0,0,0,0,0,0,NaN,NaN
8,LIB193263,2020-02-09,2020-02-10,1,94,0.979167,168.359258,29.827046,51.107337,26.038118,...,0,0,0,0,0,0,0,0,NaN,NaN


In [ ]:
# --- Train, calibrate, evaluate ---
# Patient-level split to avoid leakage
patients = dataset["Patient_ID"].unique()
rng = np.random.default_rng(42)
test_pat = set(rng.choice(patients, size=int(0.2*len(patients)), replace=False))

train_df = dataset[~dataset["Patient_ID"].isin(test_pat)].copy()
test_df  = dataset[ dataset["Patient_ID"].isin(test_pat)].copy()

y_tr = train_df["label_90d"].values
y_te = test_df["label_90d"].values

X_tr = train_df[feature_cols]
X_te = test_df[feature_cols]

# simple preprocessing (impute medians)
from sklearn.preprocessing import StandardScaler
pre = ColumnTransformer([("num", SimpleImputer(strategy="median"), feature_cols)], remainder="drop")

if HAS_XGB:
    clf = XGBClassifier(
        n_estimators=400, max_depth=4, learning_rate=0.06,
        subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
        tree_method="hist", random_state=7, eval_metric="logloss",
        scale_pos_weight=(y_tr==0).sum()/(y_tr==1).sum() if (y_tr==1).sum()>0 else 1.0
    )
else:
    from sklearn.ensemble import HistGradientBoostingClassifier
    clf = HistGradientBoostingClassifier(max_depth=3, learning_rate=0.06, max_iter=400, random_state=7)

pipe = Pipeline([("pre", pre), ("clf", clf)])
pipe.fit(X_tr, y_tr)

# Calibrate
cal = CalibratedClassifierCV(pipe, cv=3, method="isotonic")
cal.fit(X_tr, y_tr)

# Evaluate
proba = cal.predict_proba(X_te)[:,1]
auroc = roc_auc_score(y_te, proba) if len(np.unique(y_te))>1 else float("nan")
auprc = average_precision_score(y_te, proba) if len(np.unique(y_te))>1 else float("nan")
brier = brier_score_loss(y_te, proba)

thr = 0.5
pred = (proba >= thr).astype(int)
cm = confusion_matrix(y_te, pred).tolist()

print("AUROC:", auroc, "AUPRC:", auprc, "Brier:", brier, "CM@0.5:", cm)

# Save artifacts
import joblib
joblib.dump(cal, OUT / "model_calibrated.joblib")
with open(OUT / "metrics.json","w") as f:
    json.dump({"AUROC":auroc, "AUPRC":auprc, "Brier":brier, "ConfusionMatrix@0.5":cm}, f, indent=2)
with open(OUT / "feature_meta.json","w") as f:
    json.dump({"feature_cols": feature_cols}, f, indent=2)

# Save dataset for app/demo
dataset.to_parquet(OUT / "feature_dataset.parquet", index=False)

# Plots (matplotlib only)
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
fig, ax = plt.subplots()
RocCurveDisplay.from_predictions(y_te, proba, ax=ax); plt.savefig(PLOTS / "roc_curve.png"); plt.close()
fig, ax = plt.subplots()
PrecisionRecallDisplay.from_predictions(y_te, proba, ax=ax); plt.savefig(PLOTS / "pr_curve.png"); plt.close()

# Calibration plot
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(y_te, proba, n_bins=10, strategy='quantile')
plt.plot(prob_pred, prob_true, marker='o'); plt.plot([0,1],[0,1],'--')
plt.xlabel("Predicted"); plt.ylabel("Observed"); plt.title("Calibration")
plt.savefig(PLOTS / "calibration.png"); plt.close()

print("Artifacts saved in:", OUT)


AUROC: 0.940837009718153 AUPRC: 0.9508171157147218 Brier: 0.10052916014623206 CM@0.5: [[13704, 1892], [3191, 15887]]
Artifacts saved in: C:\Users\91954\Desktop\22mia1062\welldoc\outputs


In [ ]:
# --- Explainability (global + robust multi-output handling) ---
HAS_SHAP = True
try:
    import shap, matplotlib.pyplot as plt
except Exception as e:
    HAS_SHAP = False
    print("SHAP not available:", e)

if HAS_SHAP:
    # sample for speed
    sample = train_df.sample(n=min(300, len(train_df)), random_state=7)
    Xs = sample[feature_cols]

    # Try fast tree path; else fall back to generic Explainer
    exp = None
    try:
        # Try to access underlying tree model inside calibrated pipeline
        base_est = cal.estimators_[0].named_steps["clf"] if hasattr(cal, "estimators_") else None
        preproc  = cal.estimators_[0].named_steps["pre"] if hasattr(cal, "estimators_") else None
        if base_est is None or preproc is None:
            raise RuntimeError("No inner estimator/preprocessor available; using generic SHAP.")

        # transform X with the pipeline's preprocessor
        Xs_t = preproc.transform(Xs)

        # TreeExplainer on the tree model (XGB/HGB/etc.)
        tree_explainer = shap.TreeExplainer(base_est)
        sv = tree_explainer.shap_values(Xs_t)

        # Normalize outputs to a SHAP Explanation with original feature names
        if isinstance(sv, list):           # some versions return [class0, class1]
            sv_class1 = sv[1]              # choose positive class
        else:
            sv_class1 = sv                 # already 2-D
        exp = shap.Explanation(
            values=sv_class1,
            base_values=(np.mean(tree_explainer.expected_value)
                         if np.ndim(tree_explainer.expected_value)>0
                         else tree_explainer.expected_value),
            data=Xs,
            feature_names=feature_cols,
        )
    except Exception:
        # Generic Explainer on proba; may return multi-output Explanation
        exp = shap.Explainer(cal.predict_proba, Xs, feature_names=feature_cols)(Xs)

        # >>>>>>> KEY FIX: collapse multi-output to class 1 for plotting
        try:
            if hasattr(exp, "values") and getattr(exp.values, "ndim", 2) == 3:
                # shape: (n_samples, n_classes, n_features) -> pick class 1
                exp = exp[:, :, 1]
        except Exception:
            pass

    # Now exp should be 2-D (n_samples, n_features)
    shap.plots.beeswarm(exp, max_display=20, show=False)
    plt.tight_layout(); plt.savefig(PLOTS / "shap_global.png", dpi=200); plt.close()
    print("Saved global SHAP beeswarm ->", PLOTS / "shap_global.png")

    shap.plots.bar(exp, max_display=20, show=False)
    plt.tight_layout(); plt.savefig(PLOTS / "shap_bar.png", dpi=200); plt.close()
    print("Saved global SHAP bar ->", PLOTS / "shap_bar.png")
else:
    print("SHAP not installed; skipping.")


PermutationExplainer explainer: 301it [01:16,  3.40it/s]                         


Saved global SHAP beeswarm -> C:\Users\91954\Desktop\22mia1062\welldoc\outputs\plots\shap_global.png
Saved global SHAP bar -> C:\Users\91954\Desktop\22mia1062\welldoc\outputs\plots\shap_bar.png


In [ ]:
# Pick a concrete patient + their latest index_date
target_pid = dataset["Patient_ID"].iloc[0]  # replace with any ID you want

row = (dataset[dataset["Patient_ID"] == target_pid]
       .sort_values("index_date")
       .tail(1))
X1 = row[feature_cols]

local_exp = shap.Explainer(cal.predict_proba, X1, feature_names=feature_cols)(X1)
# If multi-output, reduce to class 1
if hasattr(local_exp, "values") and getattr(local_exp.values, "ndim", 2) == 3:
    local_exp = local_exp[:, :, 1]

# Waterfall = ranked top drivers for THIS prediction
shap.plots.waterfall(local_exp[0], max_display=15, show=False)
plt.tight_layout(); plt.savefig(PLOTS / f"shap_local_waterfall_{target_pid}.png", dpi=200); plt.close()

# Force = compact push/pull view
shap.plots.force(local_exp[0], matplotlib=True, show=False)
plt.tight_layout(); plt.savefig(PLOTS / f"shap_local_force_{target_pid}.png", dpi=200); plt.close()

print("Local plots saved for", target_pid)


Local plots saved for LIB193263


In [ ]:
# Global: mean |SHAP|
exp_for_table = shap.Explainer(cal.predict_proba, train_df[feature_cols], feature_names=feature_cols)(train_df[feature_cols].sample(300, random_state=7))
if hasattr(exp_for_table.values, "ndim") and exp_for_table.values.ndim == 3:
    exp_for_table = exp_for_table[:, :, 1]

global_top = (pd.DataFrame({
    "feature": feature_cols,
    "mean_abs_shap": np.abs(exp_for_table.values).mean(axis=0)
}).sort_values("mean_abs_shap", ascending=False).head(20))
global_top.to_csv(OUT / "top_global_drivers.csv", index=False)

# Local: feature contributions for the chosen patient
local_tbl = (pd.DataFrame({
    "feature": feature_cols,
    "shap_value": local_exp.values[0],
    "abs_shap": np.abs(local_exp.values[0])
}).sort_values("abs_shap", ascending=False).head(20))
local_tbl.to_csv(OUT / "top_local_drivers.csv", index=False)

print("Saved top_global_drivers.csv and top_local_drivers.csv")


PermutationExplainer explainer: 301it [01:16,  3.43it/s]                         

Saved top_global_drivers.csv and top_local_drivers.csv


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

thr_grid = np.round(np.linspace(0.1, 0.9, 9), 2)
rows = []
for t in thr_grid:
    pred = (proba >= t).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_te, pred, average="binary", zero_division=0)
    rows.append({"threshold": t, "precision": p, "recall": r, "f1": f1})
thr_df = pd.DataFrame(rows)
print(thr_df)
thr_df.to_csv(OUT / "threshold_sweep.csv", index=False)


   threshold  precision    recall        f1
0        0.1   0.688506  0.990932  0.812489
1        0.2   0.752639  0.979086  0.851057
2        0.3   0.810739  0.957648  0.878091
3        0.4   0.856770  0.911783  0.883421
4        0.5   0.893582  0.832739  0.862089
5        0.6   0.921929  0.754534  0.829874
6        0.7   0.939782  0.683877  0.791663
7        0.8   0.966623  0.584443  0.728449
8        0.9   0.990433  0.439564  0.608895


In [2]:
# Recompute test split, predict proba, and save calibration & decision curve plots
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.calibration import calibration_curve
from sklearn.metrics import roc_auc_score
import joblib

BASE = Path(".")
OUT = BASE / "outputs"; PLOTS = OUT / "plots"
DATASET_PATH = OUT / "feature_dataset.parquet"
MODEL_PATH   = OUT / "model_calibrated.joblib"

OUT.mkdir(parents=True, exist_ok=True)
PLOTS.mkdir(parents=True, exist_ok=True)

# 1) Load dataset + model
df = pd.read_parquet(DATASET_PATH)
model = joblib.load(MODEL_PATH)

# 2) Patient-level split (80/20 by Patient_ID, fixed seed)
rng = np.random.RandomState(42)
pids = np.array(sorted(df["Patient_ID"].unique()))
rng.shuffle(pids)
cut = int(0.8 * len(pids))
train_pids, test_pids = set(pids[:cut]), set(pids[cut:])

test_df  = df[df["Patient_ID"].isin(test_pids)].copy()
feature_cols = [c for c in df.columns if c not in
                ["Patient_ID","date","index_date","label_90d","a1c_prior","a1c_future"]]
X_te = test_df[feature_cols]
y_te = test_df["label_90d"].astype(int).values

# 3) Predict probabilities (fallback to predict if necessary)
try:
    proba = model.predict_proba(X_te)[:, 1]
except Exception:
    proba = model.predict(X_te).astype(float).clip(0, 1)

print("Test AUROC:", roc_auc_score(y_te, proba))

# 4) Calibration (quantile-binned)
prob_true, prob_pred = calibration_curve(y_te, proba, n_bins=10, strategy="quantile")
pd.DataFrame(
    {"bin": np.arange(1, len(prob_true)+1),
     "mean_pred": prob_pred,
     "frac_positive": prob_true}
).to_csv(OUT / "calibration_quantile.csv", index=False)

plt.figure(figsize=(5.5, 5))
plt.plot([0,1], [0,1], "--", lw=1, label="Perfectly calibrated")
plt.plot(prob_pred, prob_true, marker="o", lw=2, label="Model")
plt.xlabel("Predicted probability")
plt.ylabel("Observed fraction positive")
plt.title("Calibration (quantile-binned)")
plt.legend()
plt.tight_layout()
plt.savefig(PLOTS / "calibration_quantile.png", dpi=200)
plt.close()

# 5) Decision Curve Analysis (model vs treat-all vs treat-none)
ths = np.linspace(0.05, 0.95, 19)
nb_model, nb_all, nb_none = [], [], []
N = len(y_te)

for t in ths:
    pred = (proba >= t).astype(int)
    TP = np.sum((pred == 1) & (y_te == 1))
    FP = np.sum((pred == 1) & (y_te == 0))
    w  = t / (1 - t)  # threshold -> harm:benefit ratio
    nb_model.append((TP / N) - (FP / N) * w)

    TP_all = np.sum(y_te == 1)
    FP_all = np.sum(y_te == 0)
    nb_all.append((TP_all / N) - (FP_all / N) * w)

    nb_none.append(0.0)

nb_df = pd.DataFrame({"threshold": ths, "Model": nb_model, "Treat all": nb_all, "Treat none": nb_none})
nb_df.to_csv(OUT / "decision_curve.csv", index=False)

plt.figure(figsize=(6.5, 4.5))
plt.plot(nb_df["threshold"], nb_df["Model"], label="Model", lw=2)
plt.plot(nb_df["threshold"], nb_df["Treat all"], label="Treat all", lw=1)
plt.plot(nb_df["threshold"], nb_df["Treat none"], label="Treat none", lw=1)
plt.axhline(0, color="k", lw=0.8)
plt.xlabel("Risk threshold")
plt.ylabel("Net benefit")
plt.title("Decision Curve Analysis")
plt.legend()
plt.tight_layout()
plt.savefig(PLOTS / "decision_curve.png", dpi=200)
plt.close()

print("Saved:", PLOTS / "calibration_quantile.png", "and", PLOTS / "decision_curve.png")


Test AUROC: 0.9753551597520157
Saved: outputs\plots\calibration_quantile.png and outputs\plots\decision_curve.png
